In [1]:
%pip install numpy==2.3

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%pip install spicy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/36.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/36.3 MB ? eta -:--:--
   ------- -------------------------------- 6.8/36.3 MB 26.1 MB/s eta 0:00:02
   ------------ --------------------------- 11.3/36.3 MB 25.5 MB/s eta 0:00:01
   ------------- -------------------------- 11.8/36.3 MB 17.7 MB/s eta 0:00:02
   -------------- ------------------------- 13.1/36.3 MB 14.6 MB/s eta 0:00:02
   ----------------- ---------------------- 16.3/36.3 MB 14.8 MB/s eta 0:00:02
   ------------------------- -------------- 22.8/36.3 MB 17.3 MB/s eta 0:00:01
   ----------------------------- ---------- 27.0/36.3 MB 17.9 MB/s eta 0:00:01
   ------------------------------ --------- 28.0/36.3 MB 16.6 MB/s eta 0:00:01
   -------------------------------- ------- 29.9/36.3 MB 14.9 MB/s eta 0:00:01
   ---------------------------------- ----- 31.7/36.3 MB 14.5 MB/s eta 0


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
%pip install numba

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.3.5-cp313-cp313-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.8 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.8 MB 1.2 MB/s eta 0:00:02
   ----------- ---------------------------- 0.8/2.8 MB 1.2 MB/s eta 0:00:02
   --------------- ------------------------ 1.0/2.8 MB 1.2 MB/s eta 0:00:02
   ------------------- -------------------- 1.3/2.8 MB 1.2 MB/s eta 0:00:02
   ---------------------- ----------------- 1.6/2.8 MB 1.2 MB/s eta 0:00:02
   -------------------------- ------------- 1.8/2.8 MB 1.2 MB/s eta 0:00:01
   ------------------------------ --------- 2.1/2.8 MB 1.2 MB/s eta 0:00:01
   ---------------------------------- ----- 2.4/2.8 MB 1.2 MB/s eta 0:00:01
   ----------------------------------

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%pip install leidenalg

Defaulting to user installation because normal site-packages is not writeable
  Using cached leidenalg-0.11.0-cp38-abi3-win_amd64.whl.metadata (10 kB)
  Using cached igraph-1.0.0-cp39-abi3-win_amd64.whl.metadata (4.5 kB)
  Using cached texttable-1.7.0-py2.py3-none-any.whl.metadata (9.8 kB)
Using cached leidenalg-0.11.0-cp38-abi3-win_amd64.whl (2.0 MB)
Using cached igraph-1.0.0-cp39-abi3-win_amd64.whl (3.2 MB)
Using cached texttable-1.7.0-py2.py3-none-any.whl (10 kB)

   ------------- -------------------------- 1/3 [igraph]
   ------------- -------------------------- 1/3 [igraph]
   ------------- -------------------------- 1/3 [igraph]
   ------------- -------------------------- 1/3 [igraph]
   ------------- -------------------------- 1/3 [igraph]
   ------------- -------------------------- 1/3 [igraph]
   ------------- -------------------------- 1/3 [igraph]
   ------------- -------------------------- 1/3 [igraph]
   ------------- -------------------------- 1/3 [igraph]
   ------------


[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import networkx as nx
import pickle

In [2]:
import leidenalg

In [3]:
import igraph as ig

In [4]:
import numpy as np

In [5]:
import heapq

In [6]:
from numba import jit

In [7]:
import random

In [8]:
import time

In [9]:
from time import strftime, localtime

In [10]:
import math

In [11]:
K_SEEDS = 40
POP_SIZE = 10
MAX_GEN = 150
P_CROSSOVER = 0.6
P_MUTATION = 0.1
LAMBDA_VAL = 0.5
PROPAGATION_PROB = 0.01
MC_SIMULATIONS = 1000

In [12]:
NUM_RUNS = 1

In [13]:
def live_edge_to_edgelist(live_edge_graphs, target_nodes, p_attend):
    Gs = []
    start_poses = []
    ws = []
    Ps = []
    for h in live_edge_graphs:
        g = nx.DiGraph(h)
        # print("egdes:", g.number_of_edges())
        G = np.zeros((g.number_of_edges(), 2), dtype=np.int32)
        P = np.zeros(g.number_of_edges())
        w = np.zeros(len(g))
        print(len(list(target_nodes)))
        w[list(target_nodes)] = 1
        start_pos = np.zeros(len(g)+1, dtype=np.int32)
        curr_pos = 0
        for v in g:
            start_pos[v] = curr_pos
            for u in g.predecessors(v):
                G[curr_pos] = [u, v]
                P[curr_pos] = p_attend[u]
                curr_pos += 1
        start_pos[-1] = g.number_of_edges()
        Gs.append(G)
        Ps.append(P)
        start_poses.append(start_pos)
        ws.append(w)
    return Gs, Ps, start_poses, ws

In [14]:
@jit
def gradient_coverage_single_edgelist(x, G, P, start_pos, w, v):
    grad = np.zeros((x.shape[0]))
    #process gradient entries one node at a time
    p_all_fail = 1
    for j in range(start_pos[v], start_pos[v+1]):
        p_all_fail *= 1 - x[G[j]]*P[j]
    p_all_fail *= 1 - x[v]
    for j in range(start_pos[v], start_pos[v+1]):
        #0/0 should be 0 here
        if p_all_fail == 0:
            p_others_fail = 0
        else:
            p_others_fail = p_all_fail/(1 - x[G[j]]*P[j])
        grad[G[j]] += w[v]*P[j]*p_others_fail
    if p_all_fail > 0:
        grad[v] += w[v]*p_all_fail/(1 - x[v])
    if(np.any(np.isnan(grad))):
        print(x.min(), x.max())
        raise Exception()
    return grad

In [15]:
@jit
def gradient_estimate_all_nodes(x, Gs, Ps, start_poses, ws, B):
    '''
    Returns a stochastic estimate of the gradient of the probability that 
    every node is influenced wrt every selectable node. 
    '''
    n = start_poses[0].shape[0] - 1
    grad_est = np.zeros((x.shape[0], n))
    for b in range(B):
        v = random.randint(0, n-1)
        idx = random.randint(0, len(Gs)-1)
        grad_est[:, v] += (n/float(B)) * gradient_coverage_single_edgelist(x, Gs[idx], Ps[idx], start_poses[idx], ws[idx], v)
    return grad_est

In [16]:
@jit
def marginal_coverage_edgelist(x, G, P, start_pos, w):
    probs = np.ones(x.shape[0])
    
    for v in range(x.shape[0]):
        for j in range(start_pos[v], start_pos[v+1]):
            src = G[j][0]
            probs[v] *= 1 - x[src] * P[j]
    probs *= 1 - x
    probs = 1 - probs
    return probs

In [17]:
def make_multilinear_gradient_group(live_graphs, group_indicator, target_nodes, selectable_nodes, p_attend):
    Gs, Ps, start_poses, ws = live_edge_to_edgelist(live_graphs, target_nodes, p_attend)    
    def gradient(x, batch_size):
        x_expand = np.zeros(len(live_graphs[0]))
        x_expand[selectable_nodes] = x
        grad = gradient_estimate_all_nodes(x, Gs, Ps, start_poses, ws, batch_size) @ group_indicator
        return grad[selectable_nodes, :]
    return gradient


In [18]:
def make_multilinear_objective_samples_group(live_graphs, group_indicator, target_nodes, selectable_nodes, p_attend):
    Gs, Ps, start_poses, ws = live_edge_to_edgelist(live_graphs, target_nodes, p_attend)
    def f_all(x, batch_size):
        x_expand = np.zeros(len(live_graphs[0]))
        x_expand[selectable_nodes] = x
        pr_reached = np.zeros(len(x_expand))
        for b in range(batch_size):
            i = random.randint(0, len(Gs)-1)
            pr_reached += (1./batch_size)*marginal_coverage_edgelist(x_expand, Gs[i], Ps[i], start_poses[i], ws[i])
        return pr_reached @ group_indicator
    return f_all

In [19]:
def indicator(S, n):
    x = np.zeros(n)
    print(n)
    print(list(S))
    x[list(S)] = 1
    return x

In [20]:
def multi_to_set(f, n = None, g_nodes = None):
    if n == None:
        if g_nodes is not None:
            n = len(g_nodes)
        else:
            raise ValueError("Either n or g_nodes must be provided")
    def f_set(S):
        return f(indicator(S, n))
    return f_set

In [21]:
def greedy(items, mc, f):
    if mc >= len(items):
        S = set(items)
        return S, f(S)
    for u in items:
        print("set u", set([u]))
    upper_bounds = [(-f(set([u])), u) for u in items]    
    heapq.heapify(upper_bounds)
    starting_objective = f(set())
    S  = set()
    
    while len(S) < mc:
        val, u = heapq.heappop(upper_bounds)
        new_total = f(S.union(set([u])))
        new_val =  new_total - starting_objective
        if new_val >= -upper_bounds[0][0] - 0.01:
            S.add(u)
            starting_objective = new_total
        else:
            heapq.heappush(upper_bounds, (-new_val, u))
    return S, starting_objective

In [22]:
def sample_live_icm(g, num_graphs):
    live_edge_graphs = []
    for _ in range(num_graphs):
        h = nx.Graph()
        h.add_nodes_from(g.nodes())
        for u,v in g.edges():
            if random.random() < g[u][v]['p']:
                h.add_edge(u,v)
        live_edge_graphs.append(h)
    return live_edge_graphs

In [23]:
# def greedy_max_influence(G_sub, k, p=0.01, mc=20):
#     S = []

#     if k > len(G_sub):
#         return run_IC(G_sub, list(G_sub.nodes()), p, mc)

#     for _ in range(k):
#         best_node = None
#         max_gain = -1
#         candidates = list(G_sub.nodes())
        
#         # Nếu quá nhiều candidates, sample để tăng tốc
#         if len(candidates) > 100:
#             candidates = random.sample(candidates, 100)

#         for node in candidates:
#             if node in S:
#                 continue
#             spread = run_IC(G_sub, S + [node], p, mc=10)
#             if spread > max_gain:
#                 max_gain = spread
#                 best_node = node
        
#         if best_node:
#             S.append(best_node)
            
#     return run_IC(G_sub, S, p, mc=mc)

In [24]:
graphname = "rice_subset"

In [25]:
attributes = ["color"]

In [26]:
with open(f"dataset/{graphname}.pickle", 'rb') as f:
  G = pickle.load(f)

In [27]:

G = nx.convert_node_labels_to_integers(G, label_attribute='pid')

In [28]:
list_nodes = list(G.nodes())

In [29]:
ngIndex = { list_nodes[ni]: ni for ni in range(len(list_nodes)) }

In [30]:
for u, v in G.edges():
    G[u][v]['p'] = PROPAGATION_PROB

In [31]:
fair_vals_attr = np.zeros((NUM_RUNS, len(attributes)))
greedy_vals_attr = np.zeros((NUM_RUNS, len(attributes)))

In [32]:
include_total = False

In [33]:
group_size = {}

In [34]:
group_size[graphname] = {}

In [35]:
group_size[graphname] = {
  attribute: np.zeros((NUM_RUNS, len(np.unique([G.nodes[v][attribute]])))) for attribute in attributes
}

In [36]:
succession = True

In [37]:
ideal_influences = {}
N = len(G)

In [38]:
alpha = 0.5

In [39]:
for attribute in attributes:
  nvalues = len(np.unique([G.nodes[v][attribute] for v in G.nodes()]))
  group_size[graphname][attribute] = np.zeros((NUM_RUNS, nvalues))

In [40]:
group_size[graphname]['color'][0, 1]

np.float64(0.0)

In [41]:
def valoracle_to_single(f, i):
    def f_single(x):
        return f(x, 1000)[i]
    return f_single

In [49]:
from evolutionary import pop_init

In [50]:
for attribute_index, attribute in enumerate(attributes):
    live_graphs = sample_live_icm(G, MC_SIMULATIONS)
    group_indicator = np.ones((len(G.nodes()), 1))
    print(list(G.nodes()))
    oracle_values = make_multilinear_objective_samples_group(live_graphs, group_indicator, list(G.nodes()), list(G.nodes()), np.ones(len(G)))
    def f_multi(x):
        return oracle_values(x, 1000).sum()
    f_set = multi_to_set(f_multi, n=N, g_nodes=G.nodes())
    violation_0 = []
    violation_1 = []
    min_fraction_0 = []
    min_fraction_1 = []
    pof_0 = []
    time_0 = []
    time_1 = []
    
    for run in range(NUM_RUNS):
        print(strftime("%Y-%m-%d %H:%M:%S"))
        start_time1 = time.perf_counter()
        S, obj = greedy(list(range(len(G))), K_SEEDS, f_set)
        end_time1 = time.perf_counter()
        runningtime1 = end_time1 - start_time1
        
        start_time = time.perf_counter()
        values = np.unique([G.nodes[v][attribute] for v in G.nodes()])
        node_attributes = {}
        for vidx, val in enumerate(values):
            node_attributes[val] = [v for v in G.nodes() if G.nodes[v][attribute] == val]
            group_size[graphname][attribute][run, vidx] = len(node_attributes[val])
        
        opj_succession = {}
        if succession:
            for vidx, val in enumerate(values):
                h = nx.subgraph(G, node_attributes[val])
                h = nx.convert_node_labels_to_integers(h)
                live_graphs_h = sample_live_icm(h, 1000)
                group_indicator = np.ones((len(h.nodes()), 1))
                oracle_values = multi_to_set(valoracle_to_single(
                    make_multilinear_objective_samples_group(live_graphs_h, group_indicator, list(h.nodes()), list(h.nodes()), np.ones(len(h))),
                    0
                ), len(h))
                S_succession, opj_succession[val] = greedy(list(h.nodes()), math.ceil(len(node_attributes[val])*K_SEEDS/len(G)), oracle_values)
        
        if include_total:
            group_indicator = np.zeros((len(G.nodes()), len(values)+1))
            for value_index, value in enumerate(values):
                group_indicator[node_attributes[value], value_index] = 1
            group_indicator[:, -1] = 1
        else:
            group_indicator = np.zeros((len(G.nodes()), len(values)))
            for value_index, value in enumerate(values):
                group_indicator[node_attributes[value], value_index] = 1
        
        oracle_values = make_multilinear_objective_samples_group(
            live_graphs, group_indicator, list(G.nodes()), list(G.nodes()), np.ones(len(G))
        )
            
        f_attribute = {}
        f_multi_attribute = {}
        
        for vidx, value in enumerate(values):
            node_attributes[value] = [v for v in G.nodes() if G.nodes[v][attribute] == value]
            f_multi_attribute[value] = valoracle_to_single(oracle_values, vidx)
            f_attribute[value] = multi_to_set(f_multi_attribute[value], n=N, g_nodes=G.nodes())
        S_attribute = {}
        opt_attribute = {}
        if not succession:
            for value in values:
                S_attribute[value], opt_attribute[value] = greedy(
                    list(range(len(G))),
                    int(len(node_attributes[val])/len(G)*K_SEEDS)
                ), f_attribute[value]
    
        if succession:
            opt_attr = opj_succession
        all_opt = np.array([opt_attr[val] for val in values])
        
        def Eval(X):
            S = [ngIndex[int(i)] for i in X]
            fitness = 0
            x = np.zeros(len(G.nodes))
            x[list(S)] = 1
            
            vals = oracle_values(x, 1000)
            coverage_min = (vals / group_size[graphname][attribute][run]).min()
            violation = np.clip(all_opt - vals, 0, np.inf) / all_opt

            fitness += alpha * coverage_min
            fitness -= (1-alpha) * violation.sum() / len(values)

            return fitness     
        pop = 10
        muttation = 0.1
        crossover = 0.6
        maxgen = 150
        
        graph_ig = ig.Graph.from_networkx(G)
        
        partition = leidenalg.find_partition(graph_ig, leidenalg.ModularityVertexPartition)
        community_all_label = list(set(partition.membership))
        communties = {}
        
        # print("Label of all communities:", community_all_label)
        for community in community_all_label:
            communties[community] = [i for i, x in enumerate(partition.membership) if x == community]
        
        print("communities:", communties)
        community_label = []
        for community in list(communties.keys()):
            for cc in communties[community]:
                print(G.nodes[ngIndex[int(cc)]])
            community_label.append(list(
                set(
                    [G.nodes[ngIndex[int(cc)]][attribute] for cc in communties[community]]
                )
            ))
        pagerank = nx.pagerank(G)
        print(community_label)
        p = pop_init(POP_SIZE, K_SEEDS, communties, values, community_label, node_attributes, pagerank)
        for i in range(MAX_GEN):
            p = sorted(p, key= lambda x: Eval(x), reverse=True)
            
        

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

TypeError: unhashable type: 'list'